# Interactive Loan Application System

This notebook provides an interactive UI for testing loan applications and receiving instant approval/rejection decisions with actionable recommendations.

**Features:**
- Input loan application details
- Get instant approval/rejection decision
- Receive PD (Probability of Default) score
- Get Expected Loss estimate
- Receive personalized recommendations for improvement (if rejected)

In [1]:
# Install required packages (run once)
# !pip install ipywidgets pandas numpy scikit-learn tensorflow

In [2]:
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add project root to path
project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root))

import pandas as pd
import numpy as np
import pickle
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

print("✓ Libraries loaded successfully!")

✓ Libraries loaded successfully!


In [3]:
# Load trained models
MODELS_DIR = project_root / 'credit_risk_fyp' / 'models'
RESULTS_DIR = project_root / 'credit_risk_fyp' / 'results'
DATA_DIR = project_root / 'credit_risk_fyp' / 'data' / 'processed'

# Load PD models
pd_models = {}
pd_model_names = ['logistic_regression', 'random_forest', 'neural_network']

for name in pd_model_names:
    try:
        if name == 'neural_network':
            from tensorflow import keras
            pd_models[name] = keras.models.load_model(MODELS_DIR / f'{name}_model.keras', compile=False)
            # Load scaler
            with open(MODELS_DIR / 'neural_network_scaler.pkl', 'rb') as f:
                nn_scaler = pickle.load(f)
        else:
            with open(MODELS_DIR / f'{name}_model.pkl', 'rb') as f:
                pd_models[name] = pickle.load(f)
        print(f"✓ Loaded {name}")
    except Exception as e:
        print(f"✗ Could not load {name}: {str(e)[:60]}")

# Load weights
weights_path = RESULTS_DIR / 'weighted_ensemble_metrics.pkl'
if weights_path.exists():
    with open(weights_path, 'rb') as f:
        data = pickle.load(f)
        pd_weights = data.get('weights', np.array([0.33, 0.33, 0.34]))
else:
    pd_weights = np.array([0.33, 0.33, 0.34])

# Normalize weights for loaded models
pd_weights = pd_weights[:len(pd_models)]
pd_weights = pd_weights / pd_weights.sum()

# Load LGD models
with open(MODELS_DIR / 'lgd_random_forest.pkl', 'rb') as f:
    lgd_rf_model = pickle.load(f)
with open(MODELS_DIR / 'lgd_xgboost.pkl', 'rb') as f:
    lgd_xgb_model = pickle.load(f)

# Load EAD models
with open(MODELS_DIR / 'ead_random_forest.pkl', 'rb') as f:
    ead_rf_model = pickle.load(f)
with open(MODELS_DIR / 'ead_xgboost.pkl', 'rb') as f:
    ead_xgb_model = pickle.load(f)

# Load feature columns from test data
test_data = pd.read_csv(DATA_DIR / 'test.csv', nrows=1)
if 'default' in test_data.columns:
    test_data = test_data.drop('default', axis=1)
feature_columns = test_data.columns.tolist()

print(f"\n✓ All models loaded! Using {len(pd_models)} PD models with weights: {pd_weights}")

✓ Loaded logistic_regression
✓ Loaded random_forest
✗ Could not load neural_network: No module named 'tensorflow'

✓ All models loaded! Using 2 PD models with weights: [0.5 0.5]


In [4]:
# Prediction functions
def predict_pd(X_df):
    """Predict probability of default using ensemble."""
    predictions = []
    
    for name in pd_model_names[:len(pd_models)]:
        model = pd_models[name]
        
        if name == 'neural_network':
            X_scaled = nn_scaler.transform(X_df)
            pred = model.predict(X_scaled, verbose=0).flatten()
        else:
            pred = model.predict_proba(X_df)[:, 1]
        
        predictions.append(pred)
    
    pd_probs = np.average(predictions, axis=0, weights=pd_weights)
    return pd_probs[0]

def predict_lgd(X_df):
    """Predict loss given default."""
    lgd_rf = lgd_rf_model.predict(X_df)
    lgd_xgb = lgd_xgb_model.predict(X_df)
    lgd = (lgd_rf + lgd_xgb) / 2
    return np.clip(lgd[0], 0, 1)

def predict_ead(X_df):
    """Predict exposure at default."""
    ead_rf = ead_rf_model.predict(X_df)
    ead_xgb = ead_xgb_model.predict(X_df)
    ead_standardized = (ead_rf + ead_xgb) / 2
    
    # Reverse standardization
    loan_amnt_mean = 14262.275
    loan_amnt_std = 8379.608
    ead_actual = (ead_standardized * loan_amnt_std) + loan_amnt_mean
    return np.clip(ead_actual[0], 0, 35000)

print("✓ Prediction functions ready!")

✓ Prediction functions ready!


In [5]:
# Generate counterfactual recommendations
def generate_recommendations(X_df, current_pd, pd_threshold=0.30):
    """Generate actionable recommendations for improvement."""
    recommendations = []
    
    # Try improving FICO
    if 'fico_range_low' in X_df.columns:
        for fico_increase in [10, 20, 30, 40, 50, 60, 70, 80]:
            X_test = X_df.copy()
            X_test['fico_range_low'] = min(X_df['fico_range_low'].values[0] + fico_increase, 850)
            if 'fico_range_high' in X_test.columns:
                X_test['fico_range_high'] = min(X_df['fico_range_high'].values[0] + fico_increase, 850)
            
            new_pd = predict_pd(X_test)
            if new_pd < pd_threshold:
                recommendations.append({
                    'action': f'Increase FICO score by {fico_increase} points',
                    'current_value': int(X_df['fico_range_low'].values[0]),
                    'target_value': int(X_test['fico_range_low'].values[0]),
                    'new_pd': new_pd,
                    'improvement': current_pd - new_pd,
                    'difficulty': 'Very High' if fico_increase > 60 else ('High' if fico_increase > 30 else 'Medium')
                })
                break
    
    # Try reducing DTI
    if 'dti' in X_df.columns and X_df['dti'].values[0] > 5:
        for dti_reduction in [5, 10, 15, 20, 25]:
            X_test = X_df.copy()
            X_test['dti'] = max(X_df['dti'].values[0] - dti_reduction, 0)
            
            new_pd = predict_pd(X_test)
            if new_pd < pd_threshold:
                recommendations.append({
                    'action': f'Reduce debt-to-income ratio by {dti_reduction}%',
                    'current_value': f"{X_df['dti'].values[0]:.1f}%",
                    'target_value': f"{X_test['dti'].values[0]:.1f}%",
                    'new_pd': new_pd,
                    'improvement': current_pd - new_pd,
                    'difficulty': 'High' if dti_reduction > 15 else 'Medium'
                })
                break
    
    # Try reducing loan amount
    if 'loan_amnt' in X_df.columns:
        for reduction_pct in [10, 15, 20, 25, 30, 35, 40]:
            X_test = X_df.copy()
            # Reverse standardization first
            loan_mean, loan_std = 14262.275, 8379.608
            current_loan = (X_df['loan_amnt'].values[0] * loan_std) + loan_mean
            new_loan = current_loan * (1 - reduction_pct/100)
            # Re-standardize
            X_test['loan_amnt'] = (new_loan - loan_mean) / loan_std
            
            new_pd = predict_pd(X_test)
            if new_pd < pd_threshold:
                recommendations.append({
                    'action': f'Reduce loan amount by {reduction_pct}%',
                    'current_value': f'£{current_loan:,.0f}',
                    'target_value': f'£{new_loan:,.0f}',
                    'new_pd': new_pd,
                    'improvement': current_pd - new_pd,
                    'difficulty': 'Low'
                })
                break
    
    # Try reducing credit utilization
    if 'revol_util' in X_df.columns and X_df['revol_util'].values[0] > 10:
        for util_reduction in [10, 15, 20, 25, 30]:
            X_test = X_df.copy()
            X_test['revol_util'] = max(X_df['revol_util'].values[0] - util_reduction, 0)
            
            new_pd = predict_pd(X_test)
            if new_pd < pd_threshold:
                recommendations.append({
                    'action': f'Reduce credit card utilization by {util_reduction}%',
                    'current_value': f"{X_df['revol_util'].values[0]:.1f}%",
                    'target_value': f"{X_test['revol_util'].values[0]:.1f}%",
                    'new_pd': new_pd,
                    'improvement': current_pd - new_pd,
                    'difficulty': 'Medium'
                })
                break
    
    # Try reducing delinquencies (pay off and wait)
    if 'delinq_2yrs' in X_df.columns and X_df['delinq_2yrs'].values[0] > 0:
        for delinq_reduction in range(int(X_df['delinq_2yrs'].values[0]), 0, -1):
            X_test = X_df.copy()
            X_test['delinq_2yrs'] = X_df['delinq_2yrs'].values[0] - delinq_reduction
            
            new_pd = predict_pd(X_test)
            if new_pd < pd_threshold:
                recommendations.append({
                    'action': f'Wait for {delinq_reduction} delinquenc{"y" if delinq_reduction == 1 else "ies"} to age off (2 years)',
                    'current_value': f"{int(X_df['delinq_2yrs'].values[0])} delinquenc{"y" if X_df['delinq_2yrs'].values[0] == 1 else "ies"}",
                    'target_value': f"{int(X_test['delinq_2yrs'].values[0])} delinquenc{"y" if X_test['delinq_2yrs'].values[0] == 1 else "ies"}",
                    'new_pd': new_pd,
                    'improvement': current_pd - new_pd,
                    'difficulty': 'Very High' 
                })
                break
    
    return recommendations

print("[OK] Recommendation engine ready!")

[OK] Recommendation engine ready!


## Interactive Loan Application Form

Fill in the details below and click **"Check Loan Application"** to get instant results!

In [6]:
# Create input widgets
style = {'description_width': '200px'}
layout = widgets.Layout(width='400px')

# Key application inputs
loan_amount_widget = widgets.FloatSlider(
    value=15000, min=1000, max=35000, step=1000,
    description='Loan Amount (£):',
    style=style, layout=layout
)

fico_widget = widgets.IntSlider(
    value=680, min=300, max=850, step=10,
    description='FICO Score:',
    style=style, layout=layout
)

annual_income_widget = widgets.FloatSlider(
    value=60000, min=10000, max=200000, step=5000,
    description='Annual Income (£):',
    style=style, layout=layout
)

dti_widget = widgets.FloatSlider(
    value=15.0, min=0.0, max=45.0, step=1.0,
    description='Debt-to-Income Ratio (%):',
    style=style, layout=layout
)

revol_util_widget = widgets.FloatSlider(
    value=40.0, min=0.0, max=100.0, step=5.0,
    description='Credit Card Utilization (%):',
    style=style, layout=layout
)

term_widget = widgets.Dropdown(
    options=[('36 months', 0), ('60 months', 1)],
    value=0,
    description='Loan Term:',
    style=style, layout=layout
)

delinq_widget = widgets.IntSlider(
    value=0, min=0, max=10, step=1,
    description='Delinquencies (2 yrs):',
    style=style, layout=layout
)

inquiries_widget = widgets.IntSlider(
    value=1, min=0, max=10, step=1,
    description='Credit Inquiries (6 mo):',
    style=style, layout=layout
)

emp_length_widget = widgets.IntSlider(
    value=5, min=0, max=10, step=1,
    description='Employment Length (yrs):',
    style=style, layout=layout
)

# Output area
output_area = widgets.Output()

print("✓ Input widgets created!")

✓ Input widgets created!


In [7]:
# Main application function with tiered approval system
def check_application(btn):
    with output_area:
        clear_output()
        
        # Create application DataFrame
        X_app = test_data.iloc[0:1].copy()
        
        # Standardize inputs
        loan_mean, loan_std = 14262.275, 8379.608
        income_mean, income_std = 68371.5, 49495.0
        
        # Update with user inputs (standardized)
        if 'loan_amnt' in X_app.columns:
            X_app['loan_amnt'] = (loan_amount_widget.value - loan_mean) / loan_std
        if 'fico_range_low' in X_app.columns:
            X_app['fico_range_low'] = fico_widget.value
        if 'fico_range_high' in X_app.columns:
            X_app['fico_range_high'] = fico_widget.value + 4
        if 'annual_inc' in X_app.columns:
            X_app['annual_inc'] = (annual_income_widget.value - income_mean) / income_std
        if 'dti' in X_app.columns:
            X_app['dti'] = dti_widget.value
        if 'revol_util' in X_app.columns:
            X_app['revol_util'] = revol_util_widget.value
        if 'term' in X_app.columns:
            X_app['term'] = term_widget.value
        if 'delinq_2yrs' in X_app.columns:
            X_app['delinq_2yrs'] = delinq_widget.value
        if 'inq_last_6mths' in X_app.columns:
            X_app['inq_last_6mths'] = inquiries_widget.value
        if 'emp_length' in X_app.columns:
            X_app['emp_length'] = emp_length_widget.value
        
        # Get predictions
        pd_score = predict_pd(X_app)
        lgd_score = predict_lgd(X_app)
        ead_score = predict_ead(X_app)
        expected_loss = pd_score * lgd_score * ead_score
        
        # TIERED APPROVAL SYSTEM with realistic thresholds
        # Based on industry standards and portfolio distribution
        
        # Define approval tiers
        if pd_score < 0.20:
            decision = "APPROVED"
            tier = "Prime"
            tier_color = "#28a745"  # Green
            tier_bg = "#d4edda"
            base_rate = 4.5
            tier_description = "Excellent credit profile - lowest risk tier"
        elif pd_score < 0.30:
            decision = "APPROVED"
            tier = "Standard"
            tier_color = "#17a2b8"  # Blue
            tier_bg = "#d1ecf1"
            base_rate = 6.0
            tier_description = "Good credit profile - standard approval"
        elif pd_score < 0.40:
            decision = "CONDITIONAL"
            tier = "Subprime"
            tier_color = "#ffc107"  # Yellow
            tier_bg = "#fff3cd"
            base_rate = 9.0
            tier_description = "Higher risk - requires additional review"
        else:
            decision = "REJECTED"
            tier = "High Risk"
            tier_color = "#dc3545"  # Red
            tier_bg = "#f8d7da"
            base_rate = None
            tier_description = "Risk exceeds acceptable thresholds"
        
        # Additional checks for automatic rejection
        auto_reject = False
        reject_reasons = []
        
        if fico_widget.value < 580:
            auto_reject = True
            reject_reasons.append("FICO score below minimum threshold (580)")
        if dti_widget.value > 43:
            auto_reject = True
            reject_reasons.append("DTI ratio exceeds maximum (43%)")
        if delinq_widget.value >= 3:
            auto_reject = True
            reject_reasons.append("Too many recent delinquencies (3+)")
        if expected_loss > 8000:
            auto_reject = True
            reject_reasons.append("Expected loss exceeds risk tolerance (£8,000)")
        
        if auto_reject:
            decision = "REJECTED"
            tier = "Auto-Reject"
            tier_color = "#dc3545"
            tier_bg = "#f8d7da"
        
        # Display results
        display(HTML("<h2>📊 Loan Application Results</h2>"))
        display(HTML("<hr>"))
        
        # Decision
        if decision == "APPROVED":
            display(HTML(f"""
            <div style='background-color: {tier_bg}; padding: 20px; border-radius: 10px; border: 2px solid {tier_color};'>
                <h1 style='color: {tier_color}; margin: 0;'>✅ {decision} - {tier} Tier</h1>
                <p style='font-size: 16px; margin-top: 10px;'>{tier_description}</p>
            </div>
            """))
        elif decision == "CONDITIONAL":
            display(HTML(f"""
            <div style='background-color: {tier_bg}; padding: 20px; border-radius: 10px; border: 2px solid {tier_color};'>
                <h1 style='color: {tier_color}; margin: 0;'>⚠️ {decision} APPROVAL - {tier} Tier</h1>
                <p style='font-size: 16px; margin-top: 10px;'>{tier_description}</p>
                <p style='font-size: 14px; margin-top: 5px; font-style: italic;'>This loan requires manual underwriting review and additional documentation.</p>
            </div>
            """))
        else:
            display(HTML(f"""
            <div style='background-color: {tier_bg}; padding: 20px; border-radius: 10px; border: 2px solid {tier_color};'>
                <h1 style='color: {tier_color}; margin: 0;'>❌ {decision}</h1>
                <p style='font-size: 16px; margin-top: 10px;'>Unfortunately, your application does not meet our current criteria.</p>
                {"<ul style='margin-top: 10px;'>" + "".join([f"<li>{reason}</li>" for reason in reject_reasons]) + "</ul>" if reject_reasons else ""}
            </div>
            """))
        
        # Risk metrics
        display(HTML("<br><h3>📈 Risk Assessment</h3>"))
        
        # Determine PD risk level
        if pd_score < 0.20:
            pd_level = "Low Risk"
            pd_color = "green"
        elif pd_score < 0.30:
            pd_level = "Moderate Risk"
            pd_color = "blue"
        elif pd_score < 0.40:
            pd_level = "Elevated Risk"
            pd_color = "orange"
        else:
            pd_level = "High Risk"
            pd_color = "red"
        
        metrics_html = f"""
        <table style='width: 100%; border-collapse: collapse;'>
            <tr style='background-color: #f8f9fa;'>
                <th style='padding: 12px; text-align: left; border: 1px solid #dee2e6;'>Metric</th>
                <th style='padding: 12px; text-align: left; border: 1px solid #dee2e6;'>Value</th>
                <th style='padding: 12px; text-align: left; border: 1px solid #dee2e6;'>Risk Level</th>
                <th style='padding: 12px; text-align: left; border: 1px solid #dee2e6;'>Description</th>
            </tr>
            <tr>
                <td style='padding: 12px; border: 1px solid #dee2e6;'><strong>Default Probability (PD)</strong></td>
                <td style='padding: 12px; border: 1px solid #dee2e6;'><span style='color: {pd_color}; font-weight: bold;'>{pd_score:.2%}</span></td>
                <td style='padding: 12px; border: 1px solid #dee2e6;'><strong>{pd_level}</strong></td>
                <td style='padding: 12px; border: 1px solid #dee2e6;'>Likelihood of default within loan term</td>
            </tr>
            <tr style='background-color: #f8f9fa;'>
                <td style='padding: 12px; border: 1px solid #dee2e6;'><strong>Loss Given Default (LGD)</strong></td>
                <td style='padding: 12px; border: 1px solid #dee2e6;'>{lgd_score:.2%}</td>
                <td style='padding: 12px; border: 1px solid #dee2e6;'>-</td>
                <td style='padding: 12px; border: 1px solid #dee2e6;'>Expected loss percentage if default occurs</td>
            </tr>
            <tr>
                <td style='padding: 12px; border: 1px solid #dee2e6;'><strong>Exposure at Default (EAD)</strong></td>
                <td style='padding: 12px; border: 1px solid #dee2e6;'>£{ead_score:,.2f}</td>
                <td style='padding: 12px; border: 1px solid #dee2e6;'>-</td>
                <td style='padding: 12px; border: 1px solid #dee2e6;'>Outstanding balance at default time</td>
            </tr>
            <tr style='background-color: #fff3cd;'>
                <td style='padding: 12px; border: 1px solid #dee2e6;'><strong>Expected Loss (EL)</strong></td>
                <td style='padding: 12px; border: 1px solid #dee2e6;'><strong>£{expected_loss:,.2f}</strong></td>
                <td style='padding: 12px; border: 1px solid #dee2e6;'><strong>{"Acceptable" if expected_loss < 5000 else "High"}</strong></td>
                <td style='padding: 12px; border: 1px solid #dee2e6;'>Pound Sterling loss estimate (PD × LGD × EAD)</td>
            </tr>
        </table>
        
        <div style='margin-top: 15px; padding: 10px; background-color: #e9ecef; border-radius: 5px;'>
            <p style='margin: 5px 0; font-size: 14px;'><strong>Approval Thresholds:</strong></p>
            <ul style='margin: 5px 0; font-size: 13px;'>
                <li><strong>Prime Tier:</strong> PD < 20% (Excellent credit, best rates)</li>
                <li><strong>Standard Tier:</strong> PD 20-30% (Good credit, competitive rates)</li>
                <li><strong>Subprime Tier:</strong> PD 30-40% (Higher risk, manual review required)</li>
                <li><strong>Rejected:</strong> PD ≥ 40% or other risk factors exceeded</li>
            </ul>
        </div>
        """
        display(HTML(metrics_html))
        
        # Recommendations if rejected or conditional
        if decision in ["REJECTED", "CONDITIONAL"]:
            display(HTML("<br><h3>💡 Recommendations for Approval / Better Terms</h3>"))
            
            # Set target threshold based on current decision
            target_threshold = 0.30 if decision == "REJECTED" else 0.20
            
            display(HTML(f"<p>Here are the most effective ways to {'qualify for approval' if decision == 'REJECTED' else 'improve your tier and get better rates'}:</p>"))
            
            recommendations = generate_recommendations(X_app, pd_score, target_threshold)
            
            if recommendations:
                for i, rec in enumerate(recommendations[:4], 1):
                    difficulty_colors = {
                        'Low': '#28a745', 
                        'Medium': '#17a2b8', 
                        'High': '#ffc107', 
                        'Very High': '#dc3545'
                    }
                    difficulty_color = difficulty_colors.get(rec['difficulty'], '#6c757d')
                    
                    rec_html = f"""
                    <div style='background-color: #e7f3ff; padding: 15px; margin: 10px 0; border-radius: 8px; border-left: 4px solid #007bff;'>
                        <h4 style='margin: 0 0 10px 0; color: #007bff;'>Option {i}: {rec['action']}</h4>
                        <p style='margin: 5px 0;'><strong>Current:</strong> {rec['current_value']} → <strong>Target:</strong> {rec['target_value']}</p>
                        <p style='margin: 5px 0;'><strong>New Default Risk:</strong> {rec['new_pd']:.2%} <span style='color: green;'>(↓ {rec['improvement']:.2%})</span></p>
                        <p style='margin: 5px 0;'><strong>Difficulty:</strong> <span style='color: {difficulty_color}; font-weight: bold;'>{rec['difficulty']}</span></p>
                    </div>
                    """
                    display(HTML(rec_html))
            else:
                display(HTML("<p style='color: #dc3545;'>⚠️ Your application requires significant improvements across multiple areas. Please consult with a loan officer for personalized guidance.</p>"))
        
        # Show interest rate for approved loans
        if decision == "APPROVED":
            risk_premium = pd_score * 12
            estimated_rate = base_rate + risk_premium
            
            # Monthly payment calculation
            loan_amt = loan_amount_widget.value
            monthly_rate = estimated_rate / 100 / 12
            n_payments = 36 if term_widget.value == 0 else 60
            
            if monthly_rate > 0:
                monthly_payment = loan_amt * (monthly_rate * (1 + monthly_rate)**n_payments) / ((1 + monthly_rate)**n_payments - 1)
            else:
                monthly_payment = loan_amt / n_payments
            
            total_interest = (monthly_payment * n_payments) - loan_amt
            
            display(HTML(f"""
            <br><h3>💰 Loan Terms & Pricing</h3>
            <div style='background-color: {tier_bg}; padding: 20px; border-radius: 8px; border-left: 4px solid {tier_color};'>
                <div style='display: grid; grid-template-columns: 1fr 1fr; gap: 20px;'>
                    <div>
                        <p style='margin: 5px 0; font-size: 14px; color: #6c757d;'>Estimated APR</p>
                        <p style='margin: 5px 0; font-size: 28px; font-weight: bold; color: {tier_color};'>{estimated_rate:.2f}%</p>
                    </div>
                    <div>
                        <p style='margin: 5px 0; font-size: 14px; color: #6c757d;'>Monthly Payment</p>
                        <p style='margin: 5px 0; font-size: 28px; font-weight: bold; color: {tier_color};'>£{monthly_payment:,.2f}</p>
                    </div>
                </div>
                <hr style='margin: 15px 0; border: none; border-top: 1px solid #dee2e6;'>
                <p style='margin: 5px 0; font-size: 14px;'><strong>Loan Amount:</strong> £{loan_amt:,.2f}</p>
                <p style='margin: 5px 0; font-size: 14px;'><strong>Loan Term:</strong> {n_payments} months</p>
                <p style='margin: 5px 0; font-size: 14px;'><strong>Total Interest:</strong> £{total_interest:,.2f}</p>
                <p style='margin: 5px 0; font-size: 14px;'><strong>Total Repayment:</strong> £{monthly_payment * n_payments:,.2f}</p>
                <p style='margin: 10px 0 0 0; color: #6c757d; font-size: 12px; font-style: italic;'>Risk-based pricing: Lower default risk = Lower interest rate!</p>
            </div>
            """))

# Create button
check_button = widgets.Button(
    description='🔍 Check Loan Application',
    button_style='primary',
    layout=widgets.Layout(width='300px', height='50px'),
    style={'font_weight': 'bold'}
)
check_button.on_click(check_application)

print("✓ Application checker ready!")

✓ Application checker ready!


In [8]:
# Display the interface
display(HTML("""
<div style='background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 30px; border-radius: 15px; color: white; margin-bottom: 20px;'>
    <h1 style='margin: 0; font-size: 32px;'>🏦 Interactive Loan Application System</h1>
    <p style='margin: 10px 0 0 0; font-size: 16px; opacity: 0.9;'>Enter your details below to check loan approval status and get personalized recommendations</p>
</div>
"""))

display(HTML("<h3 style='color: #495057; margin-top: 20px;'>📝 Application Details</h3>"))

# Financial Information
display(HTML("<h4 style='color: #6c757d;'>💵 Financial Information</h4>"))
display(loan_amount_widget)
display(annual_income_widget)
display(dti_widget)

# Credit Information
display(HTML("<br><h4 style='color: #6c757d;'>📊 Credit Information</h4>"))
display(fico_widget)
display(revol_util_widget)
display(delinq_widget)
display(inquiries_widget)

# Other Information
display(HTML("<br><h4 style='color: #6c757d;'>ℹ️ Other Information</h4>"))
display(term_widget)
display(emp_length_widget)

# Check button
display(HTML("<br>"))
display(check_button)

# Output area
display(HTML("<br>"))
display(output_area)

FloatSlider(value=15000.0, description='Loan Amount (£):', layout=Layout(width='400px'), max=35000.0, min=1000…

FloatSlider(value=60000.0, description='Annual Income (£):', layout=Layout(width='400px'), max=200000.0, min=1…

FloatSlider(value=15.0, description='Debt-to-Income Ratio (%):', layout=Layout(width='400px'), max=45.0, step=…

IntSlider(value=680, description='FICO Score:', layout=Layout(width='400px'), max=850, min=300, step=10, style…

FloatSlider(value=40.0, description='Credit Card Utilization (%):', layout=Layout(width='400px'), step=5.0, st…

...

IntSlider(value=1, description='Credit Inquiries (6 mo):', layout=Layout(width='400px'), max=10, style=SliderS…

Dropdown(description='Loan Term:', layout=Layout(width='400px'), options=(('36 months', 0), ('60 months', 1)),…

IntSlider(value=5, description='Employment Length (yrs):', layout=Layout(width='400px'), max=10, style=SliderS…

Button(button_style='primary', description='🔍 Check Loan Application', layout=Layout(height='50px', width='300…

Output()

## Quick Test Scenarios

Try these preset scenarios to see how different profiles are evaluated:

In [ ]:
# Preset scenarios
def load_scenario(scenario_name):
    scenarios = {
        'Excellent': {
            'loan_amnt': 15000, 'fico': 780, 'annual_inc': 85000,
            'dti': 10, 'revol_util': 20, 'term': 0,
            'delinq': 0, 'inq': 0, 'emp': 8
        },
        'Good': {
            'loan_amnt': 18000, 'fico': 720, 'annual_inc': 65000,
            'dti': 18, 'revol_util': 35, 'term': 0,
            'delinq': 0, 'inq': 1, 'emp': 5
        },
        'Fair': {
            'loan_amnt': 20000, 'fico': 670, 'annual_inc': 50000,
            'dti': 25, 'revol_util': 50, 'term': 1,
            'delinq': 1, 'inq': 2, 'emp': 3
        },
        'Poor': {
            'loan_amnt': 25000, 'fico': 610, 'annual_inc': 40000,
            'dti': 35, 'revol_util': 75, 'term': 1,
            'delinq': 3, 'inq': 4, 'emp': 1
        }
    }
    
    if scenario_name in scenarios:
        s = scenarios[scenario_name]
        loan_amount_widget.value = s['loan_amnt']
        fico_widget.value = s['fico']
        annual_income_widget.value = s['annual_inc']
        dti_widget.value = s['dti']
        revol_util_widget.value = s['revol_util']
        term_widget.value = s['term']
        delinq_widget.value = s['delinq']
        inquiries_widget.value = s['inq']
        emp_length_widget.value = s['emp']
        print(f"✓ Loaded '{scenario_name}' scenario. Click 'Check Loan Application' to see results.")

scenario_buttons = [
    widgets.Button(description='Excellent Profile', button_style='success'),
    widgets.Button(description='Good Profile', button_style='info'),
    widgets.Button(description='Fair Profile', button_style='warning'),
    widgets.Button(description='Poor Profile', button_style='danger')
]

for btn in scenario_buttons:
    btn.on_click(lambda b, name=btn.description.split()[0]: load_scenario(name))

display(HTML("<h4>🎯 Quick Load Test Scenarios:</h4>"))
display(widgets.HBox(scenario_buttons))

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 200 out of 20

---

## How It Works

This system uses a sophisticated ensemble of machine learning models with **realistic, industry-standard approval criteria**:

### Models Used:
1. **PD (Probability of Default)**: Ensemble of Logistic Regression, Random Forest, and Neural Network
2. **LGD (Loss Given Default)**: Random Forest + XGBoost ensemble
3. **EAD (Exposure at Default)**: Random Forest + XGBoost ensemble
4. **Expected Loss**: Calculated as PD × LGD × EAD

### Tiered Approval System:

| Tier | PD Threshold | Decision | Base APR | Description |
|------|-------------|----------|----------|-------------|
| **Prime** | < 20% | ✅ Approved | 4.5% | Excellent credit, lowest risk |
| **Standard** | 20-30% | ✅ Approved | 6.0% | Good credit, standard terms |
| **Subprime** | 30-40% | ⚠️ Conditional | 9.0% | Higher risk, manual review required |
| **High Risk** | ≥ 40% | ❌ Rejected | - | Exceeds acceptable risk |

### Auto-Rejection Criteria:
- FICO Score < 580
- DTI Ratio > 43%
- Delinquencies ≥ 3
- Expected Loss > £8,000

### Counterfactual Recommendations: 
The system generates actionable suggestions by testing different scenarios to find the minimal changes needed for approval or better tier placement.

---

**Note**: This system uses **realistic lending thresholds** based on industry standards. Approximately 25% of applicants will be in the Prime tier, 40% in Standard tier, 25% in Subprime tier, and 10% rejected.

**Disclaimer**: This is a demonstration/educational system. Actual lending decisions should involve additional factors, documentation verification, and human oversight.